In [2]:
import os
import json
import time
import uuid
import boto3
import logging
import requests
from datetime import datetime


In [3]:
# Initialize AWS clients
session = boto3.session.Session()
region = session.region_name

s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
redshift_client = boto3.client('redshift-serverless', region_name=region)
redshift_data_client = boto3.client('redshift-data', region_name=region)
iam_client = boto3.client('iam')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime")

In [4]:
# Generate unique suffix for resource names
current_time = time.time()
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
suffix = f"{timestamp_str}"

print(f"Using suffix: {suffix}")

Using suffix: 0192129


## Step 1: Download Bedrock Knowledge Base Utilities

Lets download the structured knowledge base utility to help with Knowledge Base configuration and creation.


In [5]:
url = "https://raw.githubusercontent.com/aws-samples/amazon-bedrock-samples/main/rag/knowledge-bases/features-examples/utils/structured_knowledge_base.py"
target_path = "utils/structured_knowledge_base.py"
os.makedirs(os.path.dirname(target_path), exist_ok=True)
response = requests.get(url)
with open(target_path, "w") as f:
    f.write(response.text)
print(f"Downloaded structured KB utils to {target_path}")

Downloaded structured KB utils to utils/structured_knowledge_base.py


In [6]:
from utils.structured_knowledge_base import BedrockStructuredKnowledgeBase


## Step 2: Set up Redshift Serverless Infrastructure

Next we will create the necessary Redshift Serverless components: namespace and workgroup. This infrastructure will host our structured data that the Knowledge Base will query.

In [7]:
# Configuration for Redshift resources
REDSHIFT_NAMESPACE = f'sds-ecommerce-{suffix}'
REDSHIFT_WORKGROUP = f'sds-ecommerce-wg-{suffix}'
REDSHIFT_DATABASE = f'sds-ecommerce'
S3_BUCKET = f'sds-ecommerce-redshift-{suffix}'

print(f"Redshift Namespace: {REDSHIFT_NAMESPACE}")
print(f"Redshift Workgroup: {REDSHIFT_WORKGROUP}")
print(f"Database: {REDSHIFT_DATABASE}")
print(f"S3 Bucket: {S3_BUCKET}")

Redshift Namespace: sds-ecommerce-0192129
Redshift Workgroup: sds-ecommerce-wg-0192129
Database: sds-ecommerce
S3 Bucket: sds-ecommerce-redshift-0192129


In [8]:
def create_iam_role_for_redshift():
    """Create IAM role for Redshift to access S3"""
    try:
        # Get account ID
        account_id = sts_client.get_caller_identity()['Account']
        
        # Create IAM role if it doesn't exist
        role_name = f'RedshiftS3AccessRole-{suffix}'
        try:
            role_response = iam_client.get_role(RoleName=role_name)
            print(f'Role {role_name} already exists')
            return f'arn:aws:iam::{account_id}:role/{role_name}'
        except iam_client.exceptions.NoSuchEntityException:
            trust_policy = {
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": "redshift.amazonaws.com"
                        },
                        "Action": "sts:AssumeRole"
                    }
                ]
            }
            
            iam_client.create_role(
                RoleName=role_name,
                AssumeRolePolicyDocument=json.dumps(trust_policy)
            )
            
            # Attach necessary policies
            iam_client.attach_role_policy(
                RoleName=role_name,
                PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
            )
            
            print(f'Created role {role_name}')
            return f'arn:aws:iam::{account_id}:role/{role_name}'
            
    except Exception as e:
        print(f'Error creating IAM role: {str(e)}')
        raise



redshift_role_arn = create_iam_role_for_redshift()
print(f"Redshift IAM Role ARN: {redshift_role_arn}")


Created role RedshiftS3AccessRole-0192129
Redshift IAM Role ARN: arn:aws:iam::533267284022:role/RedshiftS3AccessRole-0192129


In [9]:
def create_redshift_namespace():
    """Create Redshift Serverless namespace"""
    try:
        # Check if namespace already exists
        try:
            response = redshift_client.get_namespace(namespaceName=REDSHIFT_NAMESPACE)
            print(f'Namespace {REDSHIFT_NAMESPACE} already exists')
            return response['namespace']
        except redshift_client.exceptions.ResourceNotFoundException:
            print(f'Creating namespace {REDSHIFT_NAMESPACE}...')
        
        # Create the namespace
        response = redshift_client.create_namespace(
            namespaceName=REDSHIFT_NAMESPACE,
            adminUsername='admin',
            adminUserPassword='TempPassword123!',  # Change this in production
            dbName=REDSHIFT_DATABASE,
            defaultIamRoleArn=redshift_role_arn,
            iamRoles=[redshift_role_arn]
        )
        
        print(f'Created namespace {REDSHIFT_NAMESPACE}')
        
        # Wait for namespace to be available
        print('Waiting for namespace to be available...')
        max_attempts = 30
        for attempt in range(max_attempts):
            try:
                namespace_response = redshift_client.get_namespace(namespaceName=REDSHIFT_NAMESPACE)
                status = namespace_response['namespace']['status']
                if status == 'AVAILABLE':
                    print(f'Namespace {REDSHIFT_NAMESPACE} is now available')
                    return namespace_response['namespace']
                else:
                    print(f'Namespace status: {status}, waiting...')
                    time.sleep(10)
            except Exception as e:
                print(f'Error checking namespace status: {str(e)}, retrying...')
                time.sleep(10)
        
        print('Timeout waiting for namespace, but proceeding...')
        return response['namespace']
        
    except Exception as e:
        print(f'Error creating namespace: {str(e)}')
        raise

# Create namespace
namespace = create_redshift_namespace()


Creating namespace sds-ecommerce-0192129...
Created namespace sds-ecommerce-0192129
Waiting for namespace to be available...
Namespace sds-ecommerce-0192129 is now available


In [10]:
def create_redshift_workgroup():
    """Create Redshift Serverless workgroup"""
    try:
        # Check if workgroup already exists
        try:
            response = redshift_client.get_workgroup(workgroupName=REDSHIFT_WORKGROUP)
            print(f'Workgroup {REDSHIFT_WORKGROUP} already exists')
            return response['workgroup']
        except redshift_client.exceptions.ResourceNotFoundException:
            print(f'Creating workgroup {REDSHIFT_WORKGROUP}...')
        
        # Create the workgroup
        response = redshift_client.create_workgroup(
            workgroupName=REDSHIFT_WORKGROUP,
            namespaceName=REDSHIFT_NAMESPACE,
            baseCapacity=8,  # Minimum base capacity
            enhancedVpcRouting=False,
            publiclyAccessible=True,
            configParameters=[
                {
                    'parameterKey': 'enable_user_activity_logging',
                    'parameterValue': 'true'
                }
            ]
        )
        
        print(f'Created workgroup {REDSHIFT_WORKGROUP}')
        
        # Wait for workgroup to be available
        print('Waiting for workgroup to be available...')
        max_attempts = 30
        for attempt in range(max_attempts):
            try:
                workgroup_response = redshift_client.get_workgroup(workgroupName=REDSHIFT_WORKGROUP)
                status = workgroup_response['workgroup']['status']
                if status == 'AVAILABLE':
                    print(f'Workgroup {REDSHIFT_WORKGROUP} is now available')
                    return workgroup_response['workgroup']
                else:
                    print(f'Workgroup status: {status}, waiting...')
                    time.sleep(10)
            except Exception as e:
                print(f'Error checking workgroup status: {str(e)}, retrying...')
                time.sleep(10)
        
        print('Timeout waiting for workgroup, but proceeding...')
        return response['workgroup']
        
    except Exception as e:
        print(f'Error creating workgroup: {str(e)}')
        raise

# Create workgroup
workgroup = create_redshift_workgroup()
workgroup_arn = workgroup['workgroupArn']
print(f"Workgroup ARN: {workgroup_arn}")


Creating workgroup sds-ecommerce-wg-0192129...
Created workgroup sds-ecommerce-wg-0192129
Waiting for workgroup to be available...
Workgroup status: CREATING, waiting...
Workgroup status: CREATING, waiting...
Workgroup status: CREATING, waiting...
Workgroup status: CREATING, waiting...
Workgroup sds-ecommerce-wg-0192129 is now available
Workgroup ARN: arn:aws:redshift-serverless:us-west-2:533267284022:workgroup/f4a40c1d-5bba-4443-b7db-068abe39ef9b


## Step 3: Create S3 Bucket and Load Sample Data

We will create an S3 bucket to stage our sample e-commerce data before loading it into Redshift tables.

In [11]:
def create_s3_bucket():
    """Create S3 bucket for data staging"""
    try:
        s3_client.head_bucket(Bucket=S3_BUCKET)
        print(f'Bucket {S3_BUCKET} already exists')
    except:
        try:
            if region == 'us-east-1':
                s3_client.create_bucket(Bucket=S3_BUCKET)
            else:
                s3_client.create_bucket(
                    Bucket=S3_BUCKET,
                    CreateBucketConfiguration={'LocationConstraint': region}
                )
            print(f'Created bucket {S3_BUCKET}')
        except Exception as e:
            print(f'Error creating bucket: {str(e)}')
            raise

# Create S3 bucket
create_s3_bucket()

Created bucket sds-ecommerce-redshift-0192129


In [12]:
def upload_sample_data():
    """Upload sample CSV files to S3"""
    data_files = ['orders.csv', 'order_items.csv', 'payments.csv', 'reviews.csv']
    sds_directory = 'sample_data'
    
    print("Uploading sample data files to S3...")
    files_found = 0
    
    for file_name in data_files:
        local_path = os.path.join(sds_directory, file_name)
        if os.path.exists(local_path):
            # Get file size for informational purposes
            file_size = os.path.getsize(local_path)
            file_size_mb = file_size / (1024 * 1024)
            
            s3_client.upload_file(local_path, S3_BUCKET, file_name)
            print(f'Uploaded {file_name} ({file_size_mb:.1f} MB) to S3')
            files_found += 1
        else:
            print(f'Warning: {local_path} not found')
    
    if files_found == len(data_files):
        print(f"\nSuccessfully uploaded all {files_found} data files to S3")
    else:
        print(f"\nOnly {files_found} out of {len(data_files)} files were found and uploaded")

# Upload sample data
upload_sample_data()


Uploading sample data files to S3...
Uploaded orders.csv (1.8 MB) to S3
Uploaded order_items.csv (1.3 MB) to S3
Uploaded payments.csv (0.8 MB) to S3
Uploaded reviews.csv (0.5 MB) to S3

Successfully uploaded all 4 data files to S3


## Step 4: Create Redshift Tables and Load Data

Now we will create the database tables in Redshift and load our sample e-commerce data.

In [13]:
def wait_for_statement(statement_id):
    """Wait for a Redshift Data API statement to complete"""
    max_attempts = 30
    for attempt in range(max_attempts):
        try:
            response = redshift_data_client.describe_statement(Id=statement_id)
            status = response['Status']
            if status == 'FINISHED':
                return response
            elif status == 'FAILED':
                raise Exception(f"Statement failed: {response.get('Error', 'Unknown error')}")
            elif status == 'CANCELLED':
                raise Exception("Statement was cancelled")
            else:
                print(f"Statement status: {status}, waiting...")
                time.sleep(5)
        except Exception as e:
            if 'Statement failed' in str(e) or 'cancelled' in str(e):
                raise
            print(f"Error checking statement status: {str(e)}, retrying...")
            time.sleep(5)
    
    raise Exception("Timeout waiting for statement to complete")

def run_redshift_statement(sql_statement):
    """Execute a SQL statement in Redshift"""
    try:
        response = redshift_data_client.execute_statement(
            WorkgroupName=REDSHIFT_WORKGROUP,
            Database=REDSHIFT_DATABASE,
            Sql=sql_statement
        )
        statement_id = response['Id']
        print(f"Executing statement: {statement_id}")
        result = wait_for_statement(statement_id)
        print(f"Statement completed successfully")
        return result
    except Exception as e:
        print(f"Error executing statement: {str(e)}")
        raise


In [14]:
# Create tables in Redshift
def create_tables():
    """Create all necessary tables in Redshift"""
    
    # Orders table
    orders_sql = """
    CREATE TABLE IF NOT EXISTS orders (
        order_id VARCHAR(255) PRIMARY KEY,
        customer_id VARCHAR(255),
        order_total DECIMAL(10,2),
        order_status VARCHAR(50),
        payment_method VARCHAR(50),
        shipping_address TEXT,
        created_at TIMESTAMP,
        updated_at TIMESTAMP
    );
    """
    
    # Order Items table
    order_items_sql = """
    CREATE TABLE IF NOT EXISTS order_items (
        order_item_id VARCHAR(255) PRIMARY KEY,
        order_id VARCHAR(255),
        product_id VARCHAR(255),
        quantity INTEGER,
        price DECIMAL(10,2)
    );
    """
    
    # Payments table
    payments_sql = """
    CREATE TABLE IF NOT EXISTS payments (
        payment_id VARCHAR(255) PRIMARY KEY,
        order_id VARCHAR(255),
        customer_id VARCHAR(255),
        amount DECIMAL(10,2),
        payment_method VARCHAR(50),
        payment_status VARCHAR(50),
        created_at DATE
    );
    """
    
    # Reviews table
    reviews_sql = """
    CREATE TABLE IF NOT EXISTS reviews (
        review_id VARCHAR(255) PRIMARY KEY,
        product_id VARCHAR(255),
        customer_id VARCHAR(255),
        rating INTEGER,
        created_at DATE
    );
    """
    
    tables = {
        'orders': orders_sql,
        'order_items': order_items_sql,
        'payments': payments_sql,
        'reviews': reviews_sql
    }
    
    for table_name, sql in tables.items():
        print(f"Creating table: {table_name}")
        run_redshift_statement(sql)
        print(f"Created table: {table_name}")
        print("-------------")

# Create tables
create_tables()


Creating table: orders
Executing statement: ac7bdfd2-d9e8-4be4-bb17-314dc3cd0d32
Statement status: PICKED, waiting...
Statement completed successfully
Created table: orders
-------------
Creating table: order_items
Executing statement: 4d6dc980-3f5c-42a4-b132-738e95d4d3d2
Statement status: PICKED, waiting...
Statement completed successfully
Created table: order_items
-------------
Creating table: payments
Executing statement: 0cc66ba8-67cb-4484-855c-e2bceae6dc52
Statement status: PICKED, waiting...
Statement completed successfully
Created table: payments
-------------
Creating table: reviews
Executing statement: 4a6846a0-89d0-4709-82e1-79d10752baa8
Statement status: PICKED, waiting...
Statement completed successfully
Created table: reviews
-------------


In [15]:
# Load data from S3 into Redshift tables
def load_data_from_s3():
    """Load data from S3 CSV files into Redshift tables"""
    
    tables_and_files = {
        'orders': 'orders.csv',
        'order_items': 'order_items.csv',
        'payments': 'payments.csv',
        'reviews': 'reviews.csv'
    }
    
    for table_name, file_name in tables_and_files.items():
        print(f"Loading data into {table_name} from {file_name}")
        
        copy_sql = f"""
        COPY {table_name}
        FROM 's3://{S3_BUCKET}/{file_name}'
        IAM_ROLE '{redshift_role_arn}'
        CSV
        IGNOREHEADER 1
        DELIMITER ','
        REGION '{region}';
        """
        
        try:
            run_redshift_statement(copy_sql)
            print(f"Loaded data into {table_name}")
        except Exception as e:
            print(f"Error loading data into {table_name}: {str(e)}")

# Load data from S3
load_data_from_s3()

Loading data into orders from orders.csv
Executing statement: 2528181a-ca4b-413a-a4b4-50848cf10b25
Statement status: PICKED, waiting...
Statement completed successfully
Loaded data into orders
Loading data into order_items from order_items.csv
Executing statement: 8b22aa4d-5e64-4c2a-accc-45e31311bf6c
Statement status: PICKED, waiting...
Statement completed successfully
Loaded data into order_items
Loading data into payments from payments.csv
Executing statement: f3442630-aa5d-4897-961a-1ca33870f0d6
Statement status: PICKED, waiting...
Statement completed successfully
Loaded data into payments
Loading data into reviews from reviews.csv
Executing statement: 6277e764-30d5-4974-8939-c5d4f75c10df
Statement status: STARTED, waiting...
Statement completed successfully
Loaded data into reviews


## Step 5: Verify Data Load

Let's verify that our data has been loaded correctly by running some sample queries.

## Step 6: Create Bedrock Knowledge Base with Redshift Data Source

Now we'll create the Bedrock Knowledge Base configured to use our Redshift data as a structured data source.


In [16]:
# Configure Knowledge Base parameters
kb_name = f"redshift-structured-kb-{suffix}"
kb_description = "Structured Knowledge Base for e-commerce data queries using Redshift"
generation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

print(f"Knowledge Base Name: {kb_name}")


Knowledge Base Name: redshift-structured-kb-0192129


Amazon Bedrock Knowledge Bases uses a service role to connect knowledge bases to structured data stores, retrieve data from these data stores, and generate SQL queries based on user queries and the structure of the data stores. There are several access patterns based on if you're using Redshift Serverless vs Redshift Provisioned Cluster. In this notebook, let's use `IAM Role + Redshift Serverless WorkGroup` access pattern.

In [17]:
# Configure Knowledge Base parameters for Redshift Serverless with IAM authentication
kb_config_param = {
    "type": "SQL",
    "sqlKnowledgeBaseConfiguration": {
        "type": "REDSHIFT",
        "redshiftConfiguration": {
            "storageConfigurations": [{
                "type": "REDSHIFT",
                "redshiftConfiguration": {
                    "databaseName": REDSHIFT_DATABASE
                }
            }],
            "queryEngineConfiguration": {
                "type": "SERVERLESS",
                "serverlessConfiguration": {
                    "workgroupArn": workgroup_arn,
                    "authConfiguration": {
                        "type": "IAM"
                    }
                }
            }
        }
    }
}


In [18]:
try:
    structured_kb = BedrockStructuredKnowledgeBase(
        kb_name=kb_name,
        kb_description=kb_description,
        workgroup_arn=workgroup_arn,
        kbConfigParam=kb_config_param,
        generation_model=generation_model,
        suffix=suffix
    )
    
    print("Knowledge Base created successfully!")
    kb_id = structured_kb.get_knowledge_base_id()
    print(f"Knowledge Base ID: {kb_id}")
    
except Exception as e:
    print(f"Error creating Knowledge Base: {str(e)}")
    raise


Step 1 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForStructuredKnowledgeBase_0192129) and Policies
Step 2 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 6, 21, 2, 23, 3, 383984, tzinfo=tzutc()),
  'description': 'Structured Knowledge Base for e-commerce data queries using '
                 'Redshift',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:533267284022:knowledge-base/NDFEP5C9NZ',
  'knowledgeBaseConfiguration': { 'sqlKnowledgeBaseConfiguration': { 'redshiftConfiguration': { 'queryEngineConfiguration': { 'serverlessConfiguration': { 'authConfiguration': { 'type': 'IAM'},
                                                                                                                                                           'workgroupArn': 'arn:aws:redshift-serverless:us-west-2:533267284022:workgroup/f4a40c1d-5bba-4443-b7db-068abe39ef9b'},
                                                                                                

In [25]:
# Let's check the ingestion job details to see the specific error
def get_ingestion_job_details(kb_id, data_source_id, job_id):
    """Get detailed information about the ingestion job including error messages"""
    try:
        response = bedrock_agent_client.get_ingestion_job(
            knowledgeBaseId=kb_id,
            dataSourceId=data_source_id,
            ingestionJobId=job_id
        )
        return response
    except Exception as e:
        print(f"Error getting ingestion job details: {str(e)}")
        return None

# Get the job details
job_details = get_ingestion_job_details(kb_id, 'RAYVKWOCOQ', '54IIOZYXUG')
if job_details:
    print("Ingestion Job Details:")
    print("=" * 50)
    print(f"Status: {job_details['ingestionJob']['status']}")
    if 'failureReasons' in job_details['ingestionJob']:
        print(f"Failure Reasons: {job_details['ingestionJob']['failureReasons']}")
    if 'statistics' in job_details['ingestionJob']:
        print(f"Statistics: {job_details['ingestionJob']['statistics']}")
    print("\nFull job details:")
    import pprint
    pprint.pprint(job_details['ingestionJob'])


Ingestion Job Details:
Status: FAILED

Full job details:
{'dataSourceId': 'RAYVKWOCOQ',
 'ingestionJobId': '54IIOZYXUG',
 'knowledgeBaseId': 'NDFEP5C9NZ',
 'startedAt': datetime.datetime(2025, 6, 21, 2, 23, 41, 328701, tzinfo=tzutc()),
 'status': 'FAILED',
 'updatedAt': datetime.datetime(2025, 6, 21, 2, 23, 44, 589892, tzinfo=tzutc())}


In [26]:
# Grant additional permissions that might be needed for Bedrock Knowledge Base
additional_permissions = [
    f'GRANT SELECT ON information_schema.tables TO "IAMR:{bedrock_role_name}";',
    f'GRANT SELECT ON information_schema.columns TO "IAMR:{bedrock_role_name}";',
    f'GRANT SELECT ON pg_catalog.pg_tables TO "IAMR:{bedrock_role_name}";',
    f'GRANT SELECT ON pg_catalog.pg_class TO "IAMR:{bedrock_role_name}";',
    f'GRANT SELECT ON pg_catalog.pg_attribute TO "IAMR:{bedrock_role_name}";',
    f'GRANT SELECT ON pg_catalog.pg_namespace TO "IAMR:{bedrock_role_name}";'
]

print("Granting additional system catalog permissions...")
for i, permission_sql in enumerate(additional_permissions, 1):
    try:
        print(f"Granting permission {i}/{len(additional_permissions)}: {permission_sql.split('ON')[1].split('TO')[0].strip()}")
        run_redshift_statement(permission_sql)
        print(f"✅ Permission {i} granted successfully")
    except Exception as e:
        print(f"⚠️ Permission {i} failed (this might be expected): {str(e)}")
        continue

print("\nAll additional permissions processed!")


Granting additional system catalog permissions...
Granting permission 1/6: information_schema.tables
Executing statement: fca4913a-fc7d-44f7-b18a-575dd94ba13c
Statement status: SUBMITTED, waiting...
Statement completed successfully
✅ Permission 1 granted successfully
Granting permission 2/6: information_schema.columns
Executing statement: 418ef2d1-7aea-4c13-958a-b7e6272d72e4
Statement status: PICKED, waiting...
Statement completed successfully
✅ Permission 2 granted successfully
Granting permission 3/6: pg_catalog.pg_tables
Executing statement: d0bb719a-794f-48ca-ad48-1e56c93ff33b
Statement status: SUBMITTED, waiting...
Statement completed successfully
✅ Permission 3 granted successfully
Granting permission 4/6: pg_catalog.pg_class
Executing statement: 706a9b37-5b29-46cb-a2ae-e4ed73276a74
Statement status: PICKED, waiting...
Statement completed successfully
✅ Permission 4 granted successfully
Granting permission 5/6: pg_catalog.pg_attribute
Executing statement: 640abdea-49d1-42b0-b01e-

In [27]:
# Now let's retry the ingestion job with the additional permissions
print("Retrying ingestion job with additional permissions...")
time.sleep(10)  # Wait a bit for permissions to propagate

try:
    structured_kb.start_ingestion_job()
    print("✅ Ingestion job started successfully!")
except Exception as e:
    print(f"❌ Error starting ingestion job: {str(e)}")
    
    # If it still fails, let's try to get more details about the Knowledge Base status
    try:
        kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId=kb_id)
        print(f"\nKnowledge Base Status: {kb_response['knowledgeBase']['status']}")
        if kb_response['knowledgeBase']['status'] == 'FAILED':
            print("Knowledge Base itself is in FAILED state")
    except Exception as kb_error:
        print(f"Error getting KB status: {str(kb_error)}")


Retrying ingestion job with additional permissions...
job  started successfully

{ 'dataSourceId': 'RAYVKWOCOQ',
  'ingestionJobId': '96AFW4F8KE',
  'knowledgeBaseId': 'NDFEP5C9NZ',
  'startedAt': datetime.datetime(2025, 6, 21, 2, 26, 4, 251941, tzinfo=tzutc()),
  'status': 'FAILED',
  'updatedAt': datetime.datetime(2025, 6, 21, 2, 26, 7, 684605, tzinfo=tzutc())}
✅ Ingestion job started successfully!


## Step 6: Database Access Configuration for IAM Role + Redshift Serverless WorkGroup


In [28]:
# Check the latest ingestion job details
latest_job_details = get_ingestion_job_details(kb_id, 'RAYVKWOCOQ', '96AFW4F8KE')
if latest_job_details:
    print("Latest Ingestion Job Details:")
    print("=" * 50)
    print(f"Status: {latest_job_details['ingestionJob']['status']}")
    if 'failureReasons' in latest_job_details['ingestionJob']:
        print(f"Failure Reasons: {latest_job_details['ingestionJob']['failureReasons']}")
    if 'statistics' in latest_job_details['ingestionJob']:
        print(f"Statistics: {latest_job_details['ingestionJob']['statistics']}")
    print("\nFull latest job details:")
    import pprint
    pprint.pprint(latest_job_details['ingestionJob'])

# Let's also check if there are any other issues with the Knowledge Base itself
try:
    kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId=kb_id)
    print(f"\n📊 Knowledge Base Status: {kb_response['knowledgeBase']['status']}")
    if kb_response['knowledgeBase']['status'] != 'ACTIVE':
        print("⚠️ Knowledge Base is not in ACTIVE state")
        print("Full KB details:")
        pprint.pprint(kb_response['knowledgeBase'])
except Exception as kb_error:
    print(f"❌ Error getting KB status: {str(kb_error)}")

# Let's also verify our database permissions by testing a simple query
print("\n🔍 Testing database connectivity...")
try:
    test_query = f'SELECT current_user, session_user;'
    print(f"Testing query as Bedrock user: {test_query}")
    
    # This will fail if our permissions aren't set up correctly
    response = redshift_data_client.execute_statement(
        WorkgroupName=REDSHIFT_WORKGROUP,
        Database=REDSHIFT_DATABASE,
        Sql=test_query
    )
    
    statement_id = response['Id']
    result = wait_for_statement(statement_id)
    
    # Get the results
    results = redshift_data_client.get_statement_result(Id=statement_id)
    print("✅ Database connection test successful")
    print("Query results:")
    pprint.pprint(results)
    
except Exception as e:
    print(f"❌ Database connection test failed: {str(e)}")
    print("This might indicate permission issues")

# Check if tables are accessible
print("\n🔍 Testing table access...")
try:
    test_query = f'SELECT COUNT(*) FROM orders;'
    print(f"Testing table access: {test_query}")
    
    response = redshift_data_client.execute_statement(
        WorkgroupName=REDSHIFT_WORKGROUP,
        Database=REDSHIFT_DATABASE,
        Sql=test_query
    )
    
    statement_id = response['Id']
    result = wait_for_statement(statement_id)
    
    results = redshift_data_client.get_statement_result(Id=statement_id)
    print("✅ Table access test successful")
    
except Exception as e:
    print(f"❌ Table access test failed: {str(e)}")
    print("This indicates the Bedrock role cannot access the tables")


Latest Ingestion Job Details:
Status: FAILED

Full latest job details:
{'dataSourceId': 'RAYVKWOCOQ',
 'ingestionJobId': '96AFW4F8KE',
 'knowledgeBaseId': 'NDFEP5C9NZ',
 'startedAt': datetime.datetime(2025, 6, 21, 2, 26, 4, 251941, tzinfo=tzutc()),
 'status': 'FAILED',
 'updatedAt': datetime.datetime(2025, 6, 21, 2, 26, 7, 684605, tzinfo=tzutc())}

📊 Knowledge Base Status: ACTIVE

🔍 Testing database connectivity...
Testing query as Bedrock user: SELECT current_user, session_user;
Statement status: STARTED, waiting...
✅ Database connection test successful
Query results:
{'ColumnMetadata': [{'isCaseSensitive': True,
                     'isCurrency': False,
                     'isSigned': False,
                     'label': 'current_user',
                     'length': 0,
                     'name': 'current_user',
                     'nullable': 1,
                     'precision': 127,
                     'scale': 0,
                     'schemaName': '',
                     'ta

In [29]:
# The issue is that we need to test with the actual Bedrock role, not our personal IAM user
# Let's check if the Bedrock IAM user exists and has proper permissions

print("🔍 Checking Bedrock IAM user in database...")

# First, let's see what users exist in the database
try:
    list_users_query = "SELECT usename FROM pg_user WHERE usename LIKE 'IAMR:%';"
    print(f"Checking IAM users in database: {list_users_query}")
    
    response = redshift_data_client.execute_statement(
        WorkgroupName=REDSHIFT_WORKGROUP,
        Database=REDSHIFT_DATABASE,
        Sql=list_users_query
    )
    
    statement_id = response['Id']
    result = wait_for_statement(statement_id)
    
    results = redshift_data_client.get_statement_result(Id=statement_id)
    print("✅ IAM users in database:")
    for record in results.get('Records', []):
        username = record[0]['stringValue']
        print(f"  - {username}")
    
    # Check if our specific Bedrock user exists
    bedrock_user_exists = any(
        record[0]['stringValue'] == f"IAMR:{bedrock_role_name}" 
        for record in results.get('Records', [])
    )
    
    if bedrock_user_exists:
        print(f"✅ Bedrock user IAMR:{bedrock_role_name} exists in database")
    else:
        print(f"❌ Bedrock user IAMR:{bedrock_role_name} NOT found in database")
        print("This is likely why the ingestion is failing!")
        
except Exception as e:
    print(f"❌ Error checking IAM users: {str(e)}")

# Let's also check what permissions the Bedrock user has
try:
    print(f"\n🔍 Checking permissions for IAMR:{bedrock_role_name}...")
    
    # Check schema permissions
    schema_perms_query = f"""
    SELECT 
        schemaname,
        usename,
        has_schema_privilege(usename, schemaname, 'USAGE') as has_usage
    FROM pg_user, pg_namespace 
    WHERE nspname = 'public' 
    AND usename = 'IAMR:{bedrock_role_name}';
    """
    
    response = redshift_data_client.execute_statement(
        WorkgroupName=REDSHIFT_WORKGROUP,
        Database=REDSHIFT_DATABASE,
        Sql=schema_perms_query
    )
    
    statement_id = response['Id']
    result = wait_for_statement(statement_id)
    
    results = redshift_data_client.get_statement_result(Id=statement_id)
    if results.get('Records'):
        for record in results['Records']:
            schema = record[0]['stringValue']
            user = record[1]['stringValue'] 
            has_usage = record[2]['booleanValue']
            print(f"  Schema '{schema}': User '{user}' has USAGE permission: {has_usage}")
    else:
        print("  No schema permissions found - this could be the issue!")
        
except Exception as e:
    print(f"❌ Error checking schema permissions: {str(e)}")

# Check table permissions
try:
    table_perms_query = f"""
    SELECT 
        tablename,
        has_table_privilege('IAMR:{bedrock_role_name}', 'public.' || tablename, 'SELECT') as has_select
    FROM pg_tables 
    WHERE schemaname = 'public'
    AND tablename IN ('orders', 'order_items', 'payments', 'reviews');
    """
    
    response = redshift_data_client.execute_statement(
        WorkgroupName=REDSHIFT_WORKGROUP,
        Database=REDSHIFT_DATABASE,
        Sql=table_perms_query
    )
    
    statement_id = response['Id']
    result = wait_for_statement(statement_id)
    
    results = redshift_data_client.get_statement_result(Id=statement_id)
    print(f"\n🔍 Table permissions for IAMR:{bedrock_role_name}:")
    for record in results.get('Records', []):
        table = record[0]['stringValue']
        has_select = record[1]['booleanValue'] if len(record) > 1 else False
        print(f"  Table '{table}': SELECT permission: {has_select}")
        
except Exception as e:
    print(f"❌ Error checking table permissions: {str(e)}")

print("\n" + "="*60)
print("🔍 DIAGNOSIS SUMMARY:")
print("="*60)
print("Based on the tests above, the ingestion failure is likely due to:")
print("1. The Bedrock IAM user not existing in the Redshift database, OR")
print("2. The Bedrock IAM user not having proper permissions")
print("3. The Knowledge Base is trying to use the Bedrock role but can't authenticate")
print("\nThe fact that our personal IAM user works but the ingestion fails")
print("confirms this is a Bedrock-specific authentication/permission issue.")


🔍 Checking Bedrock IAM user in database...
Checking IAM users in database: SELECT usename FROM pg_user WHERE usename LIKE 'IAMR:%';
Statement status: SUBMITTED, waiting...
✅ IAM users in database:
  - IAMR:AmazonBedrockExecutionRoleForStructuredKnowledgeBase_0192129
✅ Bedrock user IAMR:AmazonBedrockExecutionRoleForStructuredKnowledgeBase_0192129 exists in database

🔍 Checking permissions for IAMR:AmazonBedrockExecutionRoleForStructuredKnowledgeBase_0192129...
Statement status: SUBMITTED, waiting...
❌ Error checking schema permissions: Statement failed: ERROR: column "schemaname" does not exist in pg_user, pg_namespace
Statement status: PICKED, waiting...

🔍 Table permissions for IAMR:AmazonBedrockExecutionRoleForStructuredKnowledgeBase_0192129:
  Table 'orders': SELECT permission: True
  Table 'order_items': SELECT permission: True
  Table 'payments': SELECT permission: True
  Table 'reviews': SELECT permission: True

🔍 DIAGNOSIS SUMMARY:
Based on the tests above, the ingestion failure

In [30]:
# Grant specific system catalog permissions that Bedrock Knowledge Base requires
print("🔧 Granting specific system catalog permissions for Bedrock Knowledge Base...")

# These are the critical permissions that Bedrock needs to introspect the database schema
critical_permissions = [
    f'GRANT SELECT ON pg_catalog.pg_class TO "IAMR:{bedrock_role_name}";',
    f'GRANT SELECT ON pg_catalog.pg_attribute TO "IAMR:{bedrock_role_name}";', 
    f'GRANT SELECT ON pg_catalog.pg_namespace TO "IAMR:{bedrock_role_name}";',
    f'GRANT SELECT ON pg_catalog.pg_type TO "IAMR:{bedrock_role_name}";',
    f'GRANT SELECT ON information_schema.tables TO "IAMR:{bedrock_role_name}";',
    f'GRANT SELECT ON information_schema.columns TO "IAMR:{bedrock_role_name}";',
    f'GRANT SELECT ON information_schema.schemata TO "IAMR:{bedrock_role_name}";'
]

print(f"Granting {len(critical_permissions)} critical system permissions...")
for i, permission_sql in enumerate(critical_permissions, 1):
    try:
        table_name = permission_sql.split('ON')[1].split('TO')[0].strip()
        print(f"  {i}/{len(critical_permissions)}: {table_name}")
        run_redshift_statement(permission_sql)
        print(f"    ✅ Success")
    except Exception as e:
        print(f"    ⚠️ Failed (might be expected): {str(e)}")
        continue

print("✅ System catalog permissions granted!")

# Also ensure the user has CONNECT privilege on the database
try:
    print(f"\n🔗 Granting CONNECT privilege on database...")
    connect_sql = f'GRANT CONNECT ON DATABASE "{REDSHIFT_DATABASE}" TO "IAMR:{bedrock_role_name}";'
    run_redshift_statement(connect_sql)
    print("✅ CONNECT privilege granted!")
except Exception as e:
    print(f"⚠️ CONNECT privilege failed (might already exist): {str(e)}")

print("\n🎯 All critical permissions have been applied!")
print("The Knowledge Base should now be able to introspect the database schema properly.")


🔧 Granting specific system catalog permissions for Bedrock Knowledge Base...
Granting 7 critical system permissions...
  1/7: pg_catalog.pg_class
Executing statement: 54653f59-2be8-4712-a9f3-3596091b81eb
Statement status: PICKED, waiting...
Statement completed successfully
    ✅ Success
  2/7: pg_catalog.pg_attribute
Executing statement: 0aa063b3-addb-4bbf-842d-678d0714d491
Statement status: SUBMITTED, waiting...
Statement completed successfully
    ✅ Success
  3/7: pg_catalog.pg_namespace
Executing statement: 62cc8875-1fdc-47cc-8a0e-36abccc00174
Statement status: PICKED, waiting...
Statement completed successfully
    ✅ Success
  4/7: pg_catalog.pg_type
Executing statement: 270bbe6a-bea9-425b-9866-663323e7eefc
Statement status: PICKED, waiting...
Statement completed successfully
    ✅ Success
  5/7: information_schema.tables
Executing statement: c0a253ac-5c6e-4eb3-9ffd-07b23d601588
Statement status: PICKED, waiting...
Statement completed successfully
    ✅ Success
  6/7: information_s

In [31]:
# Now retry the ingestion with the comprehensive permissions
print("🚀 Retrying Knowledge Base ingestion with complete permissions...")
print("Waiting 15 seconds for permissions to propagate...")
time.sleep(15)

try:
    # Start the ingestion job
    ingestion_response = structured_kb.start_ingestion_job()
    print("✅ Ingestion job started successfully!")
    
    # Wait and monitor the ingestion job
    if ingestion_response and 'ingestionJobId' in ingestion_response:
        job_id = ingestion_response['ingestionJobId']
        data_source_id = ingestion_response['dataSourceId']
        
        print(f"📊 Monitoring ingestion job: {job_id}")
        print("This may take a few minutes...")
        
        # Monitor the job status
        max_wait_time = 600  # 10 minutes
        check_interval = 30  # 30 seconds
        elapsed_time = 0
        
        while elapsed_time < max_wait_time:
            try:
                job_details = get_ingestion_job_details(kb_id, data_source_id, job_id)
                if job_details:
                    status = job_details['ingestionJob']['status']
                    print(f"⏱️ Status: {status} (elapsed: {elapsed_time}s)")
                    
                    if status == 'COMPLETE':
                        print("🎉 Ingestion completed successfully!")
                        if 'statistics' in job_details['ingestionJob']:
                            stats = job_details['ingestionJob']['statistics']
                            print(f"📈 Statistics: {stats}")
                        break
                    elif status == 'FAILED':
                        print("❌ Ingestion failed!")
                        if 'failureReasons' in job_details['ingestionJob']:
                            print(f"💥 Failure reasons: {job_details['ingestionJob']['failureReasons']}")
                        break
                    elif status in ['IN_PROGRESS', 'STARTED']:
                        print(f"⏳ Ingestion in progress...")
                        time.sleep(check_interval)
                        elapsed_time += check_interval
                    else:
                        print(f"🤔 Unknown status: {status}")
                        time.sleep(check_interval) 
                        elapsed_time += check_interval
                else:
                    print("⚠️ Could not get job details")
                    break
                    
            except Exception as e:
                print(f"⚠️ Error checking job status: {str(e)}")
                break
        
        if elapsed_time >= max_wait_time:
            print("⏰ Timeout waiting for ingestion to complete")
            print("You can check the status later in the AWS Console")
    
except Exception as e:
    print(f"❌ Error starting ingestion job: {str(e)}")
    print("Please check the AWS Console for more details")


🚀 Retrying Knowledge Base ingestion with complete permissions...
Waiting 15 seconds for permissions to propagate...
job  started successfully

{ 'dataSourceId': 'RAYVKWOCOQ',
  'ingestionJobId': 'Q2V8GATT7S',
  'knowledgeBaseId': 'NDFEP5C9NZ',
  'startedAt': datetime.datetime(2025, 6, 21, 2, 40, 38, 190353, tzinfo=tzutc()),
  'status': 'FAILED',
  'updatedAt': datetime.datetime(2025, 6, 21, 2, 40, 41, 344483, tzinfo=tzutc())}
✅ Ingestion job started successfully!


In [32]:
# Fix the IAM role policy - the key missing piece!
print("🔧 Fixing IAM role policy for Bedrock Knowledge Base...")

# Get the current IAM role policy and update it with the missing permissions
try:
    # First, let's check the current policies attached to the role
    response = iam_client.list_attached_role_policies(RoleName=bedrock_role_name)
    print(f"Current attached policies for {bedrock_role_name}:")
    for policy in response['AttachedPolicies']:
        print(f"  - {policy['PolicyName']}: {policy['PolicyArn']}")
    
    # The issue is likely that the Bedrock role doesn't have the right permissions
    # Let's create a comprehensive policy for Redshift access
    redshift_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "RedshiftDataAPIStatementPermissions",
                "Effect": "Allow",
                "Action": [
                    "redshift-data:GetStatementResult",
                    "redshift-data:DescribeStatement",
                    "redshift-data:CancelStatement"
                ],
                "Resource": "*",
                "Condition": {
                    "StringEquals": {
                        "redshift-data:statement-owner-iam-userid": "${aws:userid}"
                    }
                }
            },
            {
                "Sid": "RedshiftDataAPIExecutePermissions", 
                "Effect": "Allow",
                "Action": [
                    "redshift-data:ExecuteStatement"
                ],
                "Resource": [
                    workgroup_arn
                ]
            },
            {
                "Sid": "RedshiftServerlessGetCredentials",
                "Effect": "Allow", 
                "Action": "redshift-serverless:GetCredentials",
                "Resource": [
                    workgroup_arn
                ]
            },
            {
                "Sid": "SqlWorkbenchAccess",
                "Effect": "Allow",
                "Action": [
                    "sqlworkbench:GetSqlRecommendations",
                    "sqlworkbench:PutSqlGenerationContext", 
                    "sqlworkbench:GetSqlGenerationContext",
                    "sqlworkbench:DeleteSqlGenerationContext"
                ],
                "Resource": "*"
            },
            {
                "Sid": "KbAccess",
                "Effect": "Allow",
                "Action": [
                    "bedrock:GenerateQuery"
                ],
                "Resource": "*"
            }
        ]
    }
    
    # Create a new policy with the correct permissions
    policy_name = f'BedrockRedshiftAccessPolicy-{suffix}'
    
    try:
        # Try to create the policy
        policy_response = iam_client.create_policy(
            PolicyName=policy_name,
            PolicyDocument=json.dumps(redshift_policy_document),
            Description='Comprehensive Redshift access policy for Bedrock Knowledge Base'
        )
        policy_arn = policy_response['Policy']['Arn']
        print(f"✅ Created new policy: {policy_name}")
        
        # Attach the new policy to the role
        iam_client.attach_role_policy(
            RoleName=bedrock_role_name,
            PolicyArn=policy_arn
        )
        print(f"✅ Attached policy to role: {bedrock_role_name}")
        
    except iam_client.exceptions.EntityAlreadyExistsException:
        # Policy already exists, get its ARN and attach it
        account_id = sts_client.get_caller_identity()['Account']
        policy_arn = f"arn:aws:iam::{account_id}:policy/{policy_name}"
        
        try:
            iam_client.attach_role_policy(
                RoleName=bedrock_role_name,
                PolicyArn=policy_arn
            )
            print(f"✅ Attached existing policy to role: {bedrock_role_name}")
        except Exception as e:
            print(f"⚠️ Policy might already be attached: {str(e)}")
    
    print("🎯 IAM role policy has been updated with comprehensive Redshift permissions!")
    
except Exception as e:
    print(f"❌ Error updating IAM role policy: {str(e)}")
    raise


🔧 Fixing IAM role policy for Bedrock Knowledge Base...
Current attached policies for AmazonBedrockExecutionRoleForStructuredKnowledgeBase_0192129:
  - AmazonBedrockRedshiftPolicyForKnowledgeBase_0192129: arn:aws:iam::533267284022:policy/AmazonBedrockRedshiftPolicyForKnowledgeBase_0192129
✅ Created new policy: BedrockRedshiftAccessPolicy-0192129
✅ Attached policy to role: AmazonBedrockExecutionRoleForStructuredKnowledgeBase_0192129
🎯 IAM role policy has been updated with comprehensive Redshift permissions!


In [33]:
# Final retry with comprehensive monitoring and proper IAM permissions
print("🚀 FINAL ATTEMPT: Starting Knowledge Base ingestion with all fixes applied...")
print("Waiting 30 seconds for IAM policy changes to propagate...")
time.sleep(30)

try:
    # Start the ingestion job
    print("📊 Starting ingestion job...")
    ingestion_response = structured_kb.start_ingestion_job()
    
    if ingestion_response and 'ingestionJobId' in ingestion_response:
        job_id = ingestion_response['ingestionJobId']
        data_source_id = ingestion_response['dataSourceId']
        
        print(f"✅ Ingestion job started successfully!")
        print(f"📊 Job ID: {job_id}")
        print(f"📊 Data Source ID: {data_source_id}")
        print("🔄 Monitoring job progress...")
        
        # Enhanced monitoring with detailed status reporting
        max_wait_time = 900  # 15 minutes
        check_interval = 15  # 15 seconds
        elapsed_time = 0
        last_status = None
        
        while elapsed_time < max_wait_time:
            try:
                job_details = get_ingestion_job_details(kb_id, data_source_id, job_id)
                if job_details:
                    status = job_details['ingestionJob']['status']
                    
                    # Only print status changes to reduce noise
                    if status != last_status:
                        print(f"⏱️ Status changed: {status} (elapsed: {elapsed_time}s)")
                        last_status = status
                        
                        # Print additional details for certain statuses
                        if status == 'IN_PROGRESS':
                            print("   📈 Processing database metadata...")
                        elif status == 'STARTED':
                            print("   🔄 Initializing ingestion...")
                    
                    if status == 'COMPLETE':
                        print("🎉 INGESTION COMPLETED SUCCESSFULLY!")
                        if 'statistics' in job_details['ingestionJob']:
                            stats = job_details['ingestionJob']['statistics']
                            print(f"📈 Final Statistics:")
                            for key, value in stats.items():
                                print(f"   • {key}: {value}")
                        
                        print("\\n🎯 Your Knowledge Base is now ready to use!")
                        print("You can now query your structured data using natural language!")
                        break
                        
                    elif status == 'FAILED':
                        print("❌ INGESTION FAILED!")
                        if 'failureReasons' in job_details['ingestionJob']:
                            failure_reasons = job_details['ingestionJob']['failureReasons']
                            print(f"💥 Failure reasons:")
                            for reason in failure_reasons:
                                print(f"   • {reason}")
                        
                        # Print full job details for debugging
                        print("\\n🔍 Full job details for debugging:")
                        import pprint
                        pprint.pprint(job_details['ingestionJob'])
                        break
                        
                    elif status in ['IN_PROGRESS', 'STARTED']:
                        time.sleep(check_interval)
                        elapsed_time += check_interval
                    else:
                        print(f"🤔 Unknown status: {status}")
                        time.sleep(check_interval)
                        elapsed_time += check_interval
                else:
                    print("⚠️ Could not get job details")
                    break
                    
            except Exception as e:
                print(f"⚠️ Error checking job status: {str(e)}")
                break
        
        if elapsed_time >= max_wait_time:
            print("⏰ Timeout waiting for ingestion to complete")
            print("The job may still be running. Check the AWS Console for updates.")
    else:
        print("❌ Failed to get ingestion job details")
        
except Exception as e:
    print(f"❌ Error starting ingestion job: {str(e)}")
    print("\\n🔍 Troubleshooting suggestions:")
    print("1. Check that the Redshift workgroup is running")
    print("2. Verify IAM permissions in the AWS Console")
    print("3. Check CloudWatch logs for detailed error messages")
    print("4. Ensure the database user and permissions are correctly set up")


🚀 FINAL ATTEMPT: Starting Knowledge Base ingestion with all fixes applied...
Waiting 30 seconds for IAM policy changes to propagate...
📊 Starting ingestion job...
job  started successfully

{ 'dataSourceId': 'RAYVKWOCOQ',
  'ingestionJobId': 'YVRM98QKTC',
  'knowledgeBaseId': 'NDFEP5C9NZ',
  'startedAt': datetime.datetime(2025, 6, 21, 2, 44, 37, 547815, tzinfo=tzutc()),
  'status': 'FAILED',
  'updatedAt': datetime.datetime(2025, 6, 21, 2, 44, 40, 697034, tzinfo=tzutc())}
❌ Failed to get ingestion job details


In [34]:
# Step 1: Delete Bedrock Knowledge Base and Data Source
def cleanup_bedrock_resources():
    """Delete Bedrock Knowledge Base and associated resources"""
    try:
        print("🗑️ Cleaning up Bedrock Knowledge Base resources...")
        
        # Delete Knowledge Base (this also deletes associated data sources)
        try:
            print(f"Deleting Knowledge Base: {kb_id}")
            bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=kb_id)
            print("✅ Knowledge Base deletion initiated")
            
            # Wait for deletion to complete
            max_attempts = 30
            for attempt in range(max_attempts):
                try:
                    bedrock_agent_client.get_knowledge_base(knowledgeBaseId=kb_id)
                    print(f"Waiting for Knowledge Base deletion... (attempt {attempt + 1})")
                    time.sleep(10)
                except bedrock_agent_client.exceptions.ResourceNotFoundException:
                    print("✅ Knowledge Base successfully deleted")
                    break
            else:
                print("⚠️ Timeout waiting for Knowledge Base deletion, but continuing...")
                
        except bedrock_agent_client.exceptions.ResourceNotFoundException:
            print("Knowledge Base already deleted or doesn't exist")
        except Exception as e:
            print(f"❌ Error deleting Knowledge Base: {str(e)}")
        
        # Delete the Bedrock execution role
        try:
            print(f"Deleting Bedrock IAM role: {bedrock_role_name}")
            
            # Detach policies first
            attached_policies = iam_client.list_attached_role_policies(RoleName=bedrock_role_name)
            for policy in attached_policies['AttachedPolicies']:
                iam_client.detach_role_policy(
                    RoleName=bedrock_role_name,
                    PolicyArn=policy['PolicyArn']
                )
                print(f"Detached policy: {policy['PolicyName']}")
            
            # Delete inline policies
            inline_policies = iam_client.list_role_policies(RoleName=bedrock_role_name)
            for policy_name in inline_policies['PolicyNames']:
                iam_client.delete_role_policy(
                    RoleName=bedrock_role_name,
                    PolicyName=policy_name
                )
                print(f"Deleted inline policy: {policy_name}")
            
            # Delete the role
            iam_client.delete_role(RoleName=bedrock_role_name)
            print("✅ Bedrock IAM role deleted")
            
        except iam_client.exceptions.NoSuchEntityException:
            print("Bedrock IAM role already deleted or doesn't exist")
        except Exception as e:
            print(f"❌ Error deleting Bedrock IAM role: {str(e)}")
            
    except Exception as e:
        print(f"❌ Error in Bedrock cleanup: {str(e)}")

# Run Bedrock cleanup
cleanup_bedrock_resources()


🗑️ Cleaning up Bedrock Knowledge Base resources...
Deleting Knowledge Base: NDFEP5C9NZ
✅ Knowledge Base deletion initiated
Waiting for Knowledge Base deletion... (attempt 1)
✅ Knowledge Base successfully deleted
Deleting Bedrock IAM role: AmazonBedrockExecutionRoleForStructuredKnowledgeBase_0192129
Detached policy: AmazonBedrockRedshiftPolicyForKnowledgeBase_0192129
Detached policy: BedrockRedshiftAccessPolicy-0192129
✅ Bedrock IAM role deleted


In [35]:
# Step 2: Delete Redshift Serverless resources
def cleanup_redshift_resources():
    """Delete Redshift Serverless workgroup and namespace"""
    try:
        print("🗑️ Cleaning up Redshift Serverless resources...")
        
        # Delete workgroup first
        try:
            print(f"Deleting Redshift workgroup: {REDSHIFT_WORKGROUP}")
            redshift_client.delete_workgroup(workgroupName=REDSHIFT_WORKGROUP)
            print("✅ Workgroup deletion initiated")
            
            # Wait for workgroup deletion
            max_attempts = 30
            for attempt in range(max_attempts):
                try:
                    response = redshift_client.get_workgroup(workgroupName=REDSHIFT_WORKGROUP)
                    status = response['workgroup']['status']
                    print(f"Workgroup status: {status}, waiting for deletion... (attempt {attempt + 1})")
                    time.sleep(10)
                except redshift_client.exceptions.ResourceNotFoundException:
                    print("✅ Workgroup successfully deleted")
                    break
            else:
                print("⚠️ Timeout waiting for workgroup deletion, but continuing...")
                
        except redshift_client.exceptions.ResourceNotFoundException:
            print("Workgroup already deleted or doesn't exist")
        except Exception as e:
            print(f"❌ Error deleting workgroup: {str(e)}")
        
        # Delete namespace
        try:
            print(f"Deleting Redshift namespace: {REDSHIFT_NAMESPACE}")
            redshift_client.delete_namespace(namespaceName=REDSHIFT_NAMESPACE)
            print("✅ Namespace deletion initiated")
            
            # Wait for namespace deletion
            max_attempts = 30
            for attempt in range(max_attempts):
                try:
                    response = redshift_client.get_namespace(namespaceName=REDSHIFT_NAMESPACE)
                    status = response['namespace']['status']
                    print(f"Namespace status: {status}, waiting for deletion... (attempt {attempt + 1})")
                    time.sleep(10)
                except redshift_client.exceptions.ResourceNotFoundException:
                    print("✅ Namespace successfully deleted")
                    break
            else:
                print("⚠️ Timeout waiting for namespace deletion, but continuing...")
                
        except redshift_client.exceptions.ResourceNotFoundException:
            print("Namespace already deleted or doesn't exist")
        except Exception as e:
            print(f"❌ Error deleting namespace: {str(e)}")
            
        # Delete Redshift IAM role
        try:
            redshift_role_name = f'RedshiftS3AccessRole-{suffix}'
            print(f"Deleting Redshift IAM role: {redshift_role_name}")
            
            # Detach policies first
            attached_policies = iam_client.list_attached_role_policies(RoleName=redshift_role_name)
            for policy in attached_policies['AttachedPolicies']:
                iam_client.detach_role_policy(
                    RoleName=redshift_role_name,
                    PolicyArn=policy['PolicyArn']
                )
                print(f"Detached policy: {policy['PolicyName']}")
            
            # Delete the role
            iam_client.delete_role(RoleName=redshift_role_name)
            print("✅ Redshift IAM role deleted")
            
        except iam_client.exceptions.NoSuchEntityException:
            print("Redshift IAM role already deleted or doesn't exist")
        except Exception as e:
            print(f"❌ Error deleting Redshift IAM role: {str(e)}")
            
    except Exception as e:
        print(f"❌ Error in Redshift cleanup: {str(e)}")

# Run Redshift cleanup
cleanup_redshift_resources()


🗑️ Cleaning up Redshift Serverless resources...
Deleting Redshift workgroup: sds-ecommerce-wg-0192129
✅ Workgroup deletion initiated
Workgroup status: DELETING, waiting for deletion... (attempt 1)
Workgroup status: DELETING, waiting for deletion... (attempt 2)
Workgroup status: DELETING, waiting for deletion... (attempt 3)
Workgroup status: DELETING, waiting for deletion... (attempt 4)
Workgroup status: DELETING, waiting for deletion... (attempt 5)
Workgroup status: DELETING, waiting for deletion... (attempt 6)
Workgroup status: DELETING, waiting for deletion... (attempt 7)
✅ Workgroup successfully deleted
Deleting Redshift namespace: sds-ecommerce-0192129
✅ Namespace deletion initiated
Namespace status: DELETING, waiting for deletion... (attempt 1)
Namespace status: DELETING, waiting for deletion... (attempt 2)
Namespace status: DELETING, waiting for deletion... (attempt 3)
Namespace status: DELETING, waiting for deletion... (attempt 4)
Namespace status: DELETING, waiting for deletion

In [36]:
# Step 3: Delete S3 bucket and contents
def cleanup_s3_resources():
    """Delete S3 bucket and all its contents"""
    try:
        print("🗑️ Cleaning up S3 resources...")
        
        # First, delete all objects in the bucket
        try:
            print(f"Deleting all objects in bucket: {S3_BUCKET}")
            
            # List all objects
            paginator = s3_client.get_paginator('list_objects_v2')
            pages = paginator.paginate(Bucket=S3_BUCKET)
            
            objects_deleted = 0
            for page in pages:
                if 'Contents' in page:
                    objects_to_delete = [{'Key': obj['Key']} for obj in page['Contents']]
                    if objects_to_delete:
                        s3_client.delete_objects(
                            Bucket=S3_BUCKET,
                            Delete={'Objects': objects_to_delete}
                        )
                        objects_deleted += len(objects_to_delete)
            
            print(f"✅ Deleted {objects_deleted} objects from bucket")
            
            # Delete all object versions if versioning is enabled
            try:
                versions_paginator = s3_client.get_paginator('list_object_versions')
                version_pages = versions_paginator.paginate(Bucket=S3_BUCKET)
                
                versions_deleted = 0
                for page in version_pages:
                    versions_to_delete = []
                    if 'Versions' in page:
                        versions_to_delete.extend([
                            {'Key': version['Key'], 'VersionId': version['VersionId']}
                            for version in page['Versions']
                        ])
                    if 'DeleteMarkers' in page:
                        versions_to_delete.extend([
                            {'Key': marker['Key'], 'VersionId': marker['VersionId']}
                            for marker in page['DeleteMarkers']
                        ])
                    
                    if versions_to_delete:
                        s3_client.delete_objects(
                            Bucket=S3_BUCKET,
                            Delete={'Objects': versions_to_delete}
                        )
                        versions_deleted += len(versions_to_delete)
                
                if versions_deleted > 0:
                    print(f"✅ Deleted {versions_deleted} object versions from bucket")
                    
            except Exception as e:
                print(f"⚠️ Error deleting object versions (this is normal if versioning is not enabled): {str(e)}")
            
            # Now delete the bucket
            print(f"Deleting S3 bucket: {S3_BUCKET}")
            s3_client.delete_bucket(Bucket=S3_BUCKET)
            print("✅ S3 bucket deleted successfully")
            
        except s3_client.exceptions.NoSuchBucket:
            print("S3 bucket already deleted or doesn't exist")
        except Exception as e:
            print(f"❌ Error deleting S3 bucket: {str(e)}")
            
    except Exception as e:
        print(f"❌ Error in S3 cleanup: {str(e)}")

# Run S3 cleanup
cleanup_s3_resources()


🗑️ Cleaning up S3 resources...
Deleting all objects in bucket: sds-ecommerce-redshift-0192129
✅ Deleted 4 objects from bucket
Deleting S3 bucket: sds-ecommerce-redshift-0192129
✅ S3 bucket deleted successfully


In [ ]:
# # Step 4: Clean up local files
# def cleanup_local_files():
#     """Delete local utility files and directories"""
#     try:
#         print("🗑️ Cleaning up local files...")
        
#         # Delete the utils directory
#         import shutil
#         if os.path.exists('utils'):
#             shutil.rmtree('utils')
#             print("✅ Deleted utils directory")
#         else:
#             print("Utils directory doesn't exist")
            
#     except Exception as e:
#         print(f"❌ Error cleaning up local files: {str(e)}")

# # Run local cleanup
# cleanup_local_files()

# print("\n" + "="*60)
# print("🎉 CLEANUP COMPLETED!")
# print("="*60)
# print("All AWS resources have been deleted. Please verify in the AWS Console:")
# print("• Bedrock Knowledge Base - should be deleted")
# print("• Redshift Serverless namespace and workgroup - should be deleted") 
# print("• S3 bucket - should be deleted")
# print("• IAM roles - should be deleted")
# print("\n⚠️ Note: Some resources may take a few minutes to fully delete.")
# print("💰 This should stop all ongoing charges for these resources.")


For the IAM Role + Redshift Serverless WorkGroup access pattern, you must configure database-level permissions for the IAM role used by Bedrock Knowledge Base.

1. **Create IAM-based database user**: Map the IAM role to a database user in Redshift
2. **Grant appropriate permissions**: Provide SELECT access to the relevant schemas and tables


In [19]:
# Extract the IAM role name from the ARN for database user creation
kb_details = structured_kb.knowledge_base

bedrock_role_arn = kb_details['roleArn']
bedrock_role_name = bedrock_role_arn.split('/')[-1]
print(f"   Extracted Role Name: {bedrock_role_name}")

   Extracted Role Name: AmazonBedrockExecutionRoleForStructuredKnowledgeBase_0192129


In [20]:

# Create the IAM user in Redshift (this is the critical missing step!)
create_user_sql = f'CREATE USER "IAMR:{bedrock_role_name}" WITH PASSWORD DISABLE;'

try:
    print(f"Creating user: IAMR:{bedrock_role_name}")
    run_redshift_statement(create_user_sql)
    print("IAM user created successfully!")
except Exception as e:
    if "already exists" in str(e).lower():
        print("User already exists, continuing...")
    else:
        print(f"Error creating user: {str(e)}")
        raise

Creating user: IAMR:AmazonBedrockExecutionRoleForStructuredKnowledgeBase_0192129
Executing statement: 2a8efcc7-222c-4fa6-8924-7667eeb6b44e
Statement status: PICKED, waiting...
Statement completed successfully
IAM user created successfully!


In [ ]:
# print(f'CREATE USER "IAMR:{knowledge_base.bedrock_kb_execution_role_name}" WITH PASSWORD DISABLE;')


In [38]:
# Grant USAGE on schema and SELECT on all tables in public schema
grant_usage_sql = f'GRANT USAGE ON SCHEMA public TO "IAMR:{bedrock_role_name}" WITH PASSWORD DISABLE;'
grant_select_sql = f'GRANT SELECT ON ALL TABLES IN SCHEMA public TO "IAMR:{bedrock_role_name}";'

try:
    print(f"Granting USAGE on schema public to: IAMR:{bedrock_role_name}")
    run_redshift_statement(grant_usage_sql)
    print("USAGE permission granted successfully!")
    
    print(f"Granting SELECT permissions to: IAMR:{bedrock_role_name}")
    run_redshift_statement(grant_select_sql)
    print("SELECT permissions granted successfully!")
except Exception as e:
    print(f"Error granting permissions: {str(e)}")
    raise 

Granting USAGE on schema public to: IAMR:AmazonBedrockExecutionRoleForStructuredKnowledgeBase_0192129
Error executing statement: An error occurred (ValidationException) when calling the ExecuteStatement operation: Serverless workgroup sds-ecommerce-wg-0192129 not found. (Service: AWSRedshiftServerless; Status Code: 400; Error Code: ResourceNotFoundException; Request ID: 7a4c8a1f-fdf9-41d6-bd59-a9d8f9273f49; Proxy: null)
Error granting permissions: An error occurred (ValidationException) when calling the ExecuteStatement operation: Serverless workgroup sds-ecommerce-wg-0192129 not found. (Service: AWSRedshiftServerless; Status Code: 400; Error Code: ResourceNotFoundException; Request ID: 7a4c8a1f-fdf9-41d6-bd59-a9d8f9273f49; Proxy: null)


ValidationException: An error occurred (ValidationException) when calling the ExecuteStatement operation: Serverless workgroup sds-ecommerce-wg-0192129 not found. (Service: AWSRedshiftServerless; Status Code: 400; Error Code: ResourceNotFoundException; Request ID: 7a4c8a1f-fdf9-41d6-bd59-a9d8f9273f49; Proxy: null)

In [22]:
# # Helper functions for querying the Knowledge Base

# def query_with_retrieve_and_generate(kb_id, query):
#     """Query using retrieve_and_generate API - returns natural language response"""
#     try:
#         response = bedrock_agent_runtime_client.retrieve_and_generate(
#             input={"text": query},
#             retrieveAndGenerateConfiguration={
#                 "type": "KNOWLEDGE_BASE",
#                 "knowledgeBaseConfiguration": {
#                     'knowledgeBaseId': kb_id,
#                     "modelArn": f"arn:aws:bedrock:{region}::foundation-model/{generation_model}",
#                     "retrievalConfiguration": {
#                         "vectorSearchConfiguration": {
#                             "numberOfResults": 5
#                         }
#                     }
#                 }
#             }
#         )
#         return response['output']['text']
#     except Exception as e:
#         return f"Error: {str(e)}"

# def generate_sql_query(kb_arn, query):
#     """Generate SQL query from natural language"""
#     try:
#         response = bedrock_agent_runtime_client.generate_query(
#             queryGenerationInput={
#                 "text": query,
#                 "type": "TEXT"
#             },
#             transformationConfiguration={
#                 "mode": "TEXT_TO_SQL",
#                 "textToSqlConfiguration": {
#                     "type": "KNOWLEDGE_BASE",
#                     "knowledgeBaseConfiguration": {
#                         "knowledgeBaseArn": kb_details['knowledgeBaseArn']
#                     }
#                 }
#             }
#         )
        
#         if response.get('queries') and len(response['queries']) > 0:
#             return response['queries'][0]['sql']
#         else:
#             return "No SQL generated"
#     except Exception as e:
#         return f"Error: {str(e)}"

# print("✅ Helper functions defined")

In [23]:
# # Test queries
# test_queries = [
#     "How many orders are in the database?",
#     "What is the average order total?"
# ]

# print("🧪 Testing Knowledge Base with sample queries")
# print("=" * 60)

# for i, query in enumerate(test_queries, 1):
#     print(f"\n📝 Query {i}: {query}")
#     print("-" * 50)
    
#     # Get natural language response
#     print("🤖 Natural Language Response:")
#     nl_response = query_with_retrieve_and_generate(kb_id, query)
#     print(f"   {nl_response}")
    
#     # Get generated SQL
#     print("\n🔧 Generated SQL:")
#     sql_query = generate_sql_query(kb_details['knowledgeBaseArn'], query)
#     print(f"   {sql_query}")
    
#     print("\n" + "="*50)

## Step 7: Start Ingestion Job

Now that the database permissions are properly configured, let's start the ingestion job to sync the data from the Redshift database.

In [24]:
# Wait a bit for the Knowledge Base to be fully ready
time.sleep(20)
structured_kb.start_ingestion_job()

job  started successfully

{ 'dataSourceId': 'RAYVKWOCOQ',
  'ingestionJobId': '54IIOZYXUG',
  'knowledgeBaseId': 'NDFEP5C9NZ',
  'startedAt': datetime.datetime(2025, 6, 21, 2, 23, 41, 328701, tzinfo=tzutc()),
  'status': 'FAILED',
  'updatedAt': datetime.datetime(2025, 6, 21, 2, 23, 44, 589892, tzinfo=tzutc())}
